22H1120016 Trần Đăng Nam

Khai thác mẫu phổ biến và kết hợp 

Lab 5.1

Cài đặt giải thuật Apriori trên Python, và chạy thử nghiệm với dữ liệu Market-Basket ở slide số 3 trong buổi lý thuyết, (chọn minSup và minConf phù hợp).

Yêu cầu xuất ra các tập phổ biến và luật kết hợp và tham số support/confidence của chúng.

In [3]:
from itertools import combinations
import pandas as pd

# Bước 1: Bảng transaction 
transactions = [
    ['Bread', 'Milk'],
    ['Bread', 'Diaper', 'Beer', 'Eggs'],
    ['Milk', 'Diaper', 'Beer', 'Coke'],
    ['Bread', 'Milk', 'Diaper', 'Beer'],
    ['Bread', 'Milk', 'Diaper', 'Coke']
]

# Tham số của Mẫu phổ biến 
minSup = 0.4  # Minimum support (40%)
minConf = 0.8  # Minimum confidence (80%)
total_transactions = len(transactions)

In [4]:
# Bước 2: lấy các item "unique" 
items = set()
for transaction in transactions:
    for item in transaction:
        items.add(item)
items = sorted(items)

In [5]:
# Bước 3: Tạo mẫu phổ biến "frequency" - Apriori
def get_support(itemset, transactions):
    count = sum(1 for transaction in transactions if set(itemset).issubset(transaction))
    return count / total_transactions

# Initialize frequent itemsets
frequent_itemsets = {}
k = 1
candidates = [[item] for item in items]

while candidates:
    # Tính "support" của từng candidate
    frequent = []
    for candidate in candidates:
        support = get_support(candidate, transactions)
        if support >= minSup:
            frequent.append((tuple(candidate), support))
    
    if not frequent:
        break
    
    frequent_itemsets[k] = frequent
    k += 1
    
    # Tạo candidates cho vòng lặp kế (để check) 
    candidates = []
    for i in range(len(frequent)):
        for j in range(i + 1, len(frequent)):
            itemset1 = list(frequent[i][0])
            itemset2 = list(frequent[j][0])
            if itemset1[:-1] == itemset2[:-1]:
                new_candidate = sorted(list(set(itemset1 + itemset2)))
                if len(new_candidate) == k:
                    candidates.append(new_candidate)

# Bước 4: Tạo quy luật kết hợp
association_rules = []
for k, itemsets in frequent_itemsets.items():
    if k < 2:
        continue
    for itemset, support in itemsets:
        itemset = set(itemset)
        for r in range(1, len(itemset)):
            for antecedent in combinations(itemset, r):
                antecedent = set(antecedent)
                consequent = itemset - antecedent
                if not consequent:
                    continue
                conf = get_support(itemset, transactions) / get_support(antecedent, transactions)
                if conf >= minConf:
                    association_rules.append((antecedent, consequent, support, conf))



In [6]:
# Bước 5 hiển thị quy luật
print("Frequent Itemsets:")
for k, itemsets in frequent_itemsets.items():
    for itemset, support in itemsets:
        print(f"Itemset: {set(itemset)}, Support: {support:.2f}")

print("\nAssociation Rules:")
for antecedent, consequent, support, conf in association_rules:
    print(f"Rule: {set(antecedent)} -> {set(consequent)}, Support: {support:.2f}, Confidence: {conf:.2f}")

Frequent Itemsets:
Itemset: {'Beer'}, Support: 0.60
Itemset: {'Bread'}, Support: 0.80
Itemset: {'Coke'}, Support: 0.40
Itemset: {'Diaper'}, Support: 0.80
Itemset: {'Milk'}, Support: 0.80
Itemset: {'Beer', 'Bread'}, Support: 0.40
Itemset: {'Beer', 'Diaper'}, Support: 0.60
Itemset: {'Beer', 'Milk'}, Support: 0.40
Itemset: {'Bread', 'Diaper'}, Support: 0.60
Itemset: {'Milk', 'Bread'}, Support: 0.60
Itemset: {'Coke', 'Diaper'}, Support: 0.40
Itemset: {'Coke', 'Milk'}, Support: 0.40
Itemset: {'Milk', 'Diaper'}, Support: 0.60
Itemset: {'Beer', 'Bread', 'Diaper'}, Support: 0.40
Itemset: {'Beer', 'Diaper', 'Milk'}, Support: 0.40
Itemset: {'Milk', 'Bread', 'Diaper'}, Support: 0.40
Itemset: {'Coke', 'Diaper', 'Milk'}, Support: 0.40

Association Rules:
Rule: {'Beer'} -> {'Diaper'}, Support: 0.60, Confidence: 1.00
Rule: {'Coke'} -> {'Diaper'}, Support: 0.40, Confidence: 1.00
Rule: {'Coke'} -> {'Milk'}, Support: 0.40, Confidence: 1.00
Rule: {'Beer', 'Bread'} -> {'Diaper'}, Support: 0.40, Confidence